In [4]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import numpy as np
import onnx
import json
from onnx2pytorch import ConvertModel

In [2]:

TRAIN_BS = 32
TEST_BS = 32

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
        ])

DLD_DATA = True
train_set = datasets.MNIST('./data', train=True, download=DLD_DATA,
                          transform=transform)
test_set = datasets.MNIST('./data', train=False, download=DLD_DATA,
                          transform=transform)
X_train_tensor = train_set.data
y_train_tensor = train_set.targets
X_test_tensor = test_set.data
y_test_tensor = test_set.targets
train_loader = DataLoader(train_set, batch_size=1, shuffle=True)
test_loader = DataLoader(test_set, batch_size=1, shuffle=True)
num_labels = len(np.unique(test_set.targets))
num_inputs = len(X_train_tensor[0])

In [3]:
random_seed=42
epsilons = [0, .05, .1, .15, .2, .25, .3]
# Set random seed for reproducibility
torch.manual_seed(random_seed)

def get_relu_activations(model, input):
  model_relu_layers=[]
  i=0
  for name, layer in model.named_modules():
    if i>0:
      input=layer(input)
      if isinstance(layer, nn.ReLU):
        model_relu_layers.append(input[0]) #if using unsqeeze, use [0].
    i=i+1
  concatenated_tensor=torch.concatenate(model_relu_layers)
  concatenated_tensor.flatten()
  return concatenated_tensor


def get_binary_abstraction(activations):
  nap=[]
  for i in activations:
    if i>0:
      nap.append(1)
    elif i==0:
      nap.append(0)
    else:
      nap.append('*')
  return nap


def find_states(onnx_path=None, states_path="wbc_480_relu_states.txt"):
  global neurons, relu_layers

  neurons=0
  relu_layers=0
  # Assuming the ReLU layers are named similarly to your PyTorch model, you can add hooks like this
  for name, layer in model.named_modules():
    if isinstance(layer, nn.Linear):
      neurons=layer.in_features
    if isinstance(layer, torch.nn.ReLU):
      relu_layers+=1

  states={}
  for i in range(num_labels):
    states[i]=[]

  with torch.no_grad():
    for i, t in train_loader:
        t=t.item()
        i=i.float().to(device)
        out=get_relu_activations(model,i)
        states[t].append(out[0].detach().cpu().numpy().tolist())

  with open(states_path, "w") as fp:
    json.dump(states, fp)

  return states

def get_label_naps(states,delta=0.99):
  label_naps={}
  for label in states:
    label_naps[label]={}
    for relu in range(len(states[label][0])):
      if (np.count_nonzero(states[label][:,relu])/len(states[label][:,relu]))>=delta:
        label_naps[label][relu]=1
      elif (np.count_nonzero(states[label][:,relu])/len(states[label][:,relu])) <=(1-delta):
        label_naps[label][relu]=0
  return label_naps


In [4]:
model = ConvertModel(onnx.load('mnist-net_256x4.onnx')).to(device)

/home/saimadhavang/.local/lib/python3.10/site-packages/onnx2pytorch/convert/layer.py:30: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at ../torch/csrc/utils/tensor_numpy.cpp:206.)
  layer.weight.data = torch.from_numpy(numpy_helper.to_array(weight))


In [5]:
# images, labels = next(iter(train_loader))
# images
neurons=0
relu_layers=0
# Assuming the ReLU layers are named similarly to your PyTorch model, you can add hooks like this
for name, layer in model.named_modules():
  if isinstance(layer, nn.Linear):
    neurons=layer.in_features
  if isinstance(layer, torch.nn.ReLU):
    relu_layers+=1

states={}
for i in range(num_labels):
  states[i]=[]

with torch.no_grad():
  for i, t in train_loader:
      t=t.item()
      i=i.float().to(device)
      out=get_relu_activations(model,i)
      states[t].append(out.detach().cpu().numpy().tolist())
print(len(states))
print(len(states[0]))

10
5923


In [12]:
states[0].shape

(5923, 1024)

In [6]:
for i in states:
  states[i]=np.array(states[i])
label_naps = get_label_naps(states)

In [7]:
i=0
neurons=0
# Assuming the ReLU layers are named similarly to your PyTorch model, you can add hooks like this
for name, layer in model.named_modules():
  if isinstance(layer, nn.Linear):
    neurons=layer.in_features
  if isinstance(layer, torch.nn.ReLU):
      i=i+1
relu_layers=i

In [1]:
import pickle

In [10]:
pickle.dump(label_naps, open("./label_naps_99.pkl", "wb"))

In [2]:
label_naps = pickle.load(open("./label_naps_99.pkl", "rb"))

In [11]:
jj = json.load(open("./NAPs/256x4_delta99_nt.json"))

In [34]:
jj['0']['A']['indices']

[[0, 105], [0, 147], [0, 159]]

In [47]:
from copy import deepcopy

In [48]:
ii = deepcopy(label_naps)

In [49]:
for label in range(10):
    for idx in jj[str(label)]['A']['indices']:
        new_idx = idx[0]*256 + idx[1]
        assert ii[label][new_idx] == 1
        del ii[label][new_idx]
    for idx in jj[str(label)]['B']['indices']:
        new_idx = idx[0]*256 + idx[1]
        assert ii[label][new_idx] == 0
        del ii[label][new_idx]
    assert len(ii[label]) ==0

In [50]:
ii

{0: {}, 1: {}, 2: {}, 3: {}, 4: {}, 5: {}, 6: {}, 7: {}, 8: {}, 9: {}}

In [22]:
for idx in jj['0']['A']['indices']:
    new_idx = idx[0]*256 + idx[1]
    assert ii[0][new_idx] == 1
    del ii[0][new_idx]

In [24]:
for idx in jj['0']['B']['indices']:
    new_idx = idx[0]*256 + idx[1]
    assert ii[0][new_idx] == 0
    del ii[0][new_idx]

In [25]:
len(ii[0])

0

In [21]:
jj['0']['A']['len'] + jj['0']['B']['len']

620